# Face Area Calculations

## Overview

This notebook will showcase the different area calculation options provided by UXarray

For more details on how to load in data, check out our [previous usage
example](https://uxarray.readthedocs.io/en/latest/examples/001-read-grid-data.html)

**This notebook has the following sections:**
1. Compute total area of all faces
2. Options for accuracy of area calculation functions
3. Getting area of all the faces (not total)
4. Calculate area of a single triangle
5. Calculate area of multiple vertices in spherical coordinates
6. Calculate area with points, without loading a UXarray grid


We will be using the `outCSne30.ug` grid file which is encoded in the UGRID convention.
Let us first read in the data:

In [ ]:
import uxarray as ux
import numpy as np
import xarray as xr

In [ ]:
# Base data path
base_path = "../../test/meshfiles/"

# Path to Grid files
ugrid_path = base_path + "/ugrid/outCSne30/outCSne30.ug"

# Load grid files and create UXarray Grid objects
ugrid_ds = xr.open_dataset(ugrid_path)

ugrid = ux.Grid(ugrid_ds)
ugrid.ds

## 1. Calculate total area of all faces
By calling the function `calculate_total_face_area()`

In [ ]:
# Calculate total area using the ugrid object
t4_area = ugrid.calculate_total_face_area()
t4_area
# 4*Pi is the area 4*3.14 = 12.56

## 2. Options for `Grid.calculate_total_face_area` functions

Quadrature Rule: gaussian or triangular

Order: 

       1 to 10              for gaussian 

       1, 4, 8, 10 and 12   for triangular

Default: triangular and order 4

In [ ]:

t1_area = ugrid.calculate_total_face_area(quadrature_rule="triangular", order=1)
t1_area

For the above result: notice that the area is different from the first mesh above

Now we use "triangular" quadrature rule and order 1

This is less time consuming as the order is only 1. 
Generally, gaussian quadrature rule is more accurate than the triangular quadrature rule. Also higher the order -> higher the time taken for area computation -> higher the accuracy. See `uxarray/get_quadratureDG.py` file for details on quadrature points and weights.

## 3. Getting area of all the faces (not total)

We use the Grid property `face_area` to get the area of all the faces in the mesh. 

Calling `face_area` again will use the cached value.


In [ ]:
ugrid.face_areas

Now calculate the area again with `compute face_areas` function
pass arguments: quadrature_rule "gaussian" and order 4

In [ ]:
all_face_areas = ugrid.compute_face_areas(quadrature_rule="gaussian", order=4)
g4_area = all_face_areas.sum()
g4_area

Now we compare the values with actual know value and report error for each of the three cases above. Clearly, quadrature_rule="gaussian", order=4 is the most accurate and quadrature_rule="triangular", order=1 is the least.

In [ ]:
actual_area = 4 * np.pi
diff_t4_area = np.abs(t4_area - actual_area)
diff_t1_area = np.abs(t1_area - actual_area)
diff_g4_area = np.abs(g4_area - actual_area)

diff_t1_area, diff_t4_area, diff_g4_area

## 4. Calculate area of a single triangle (cartesian coordinates)

For this section, we define a new 1 triangle face grid with 3 vertices

Assume the units in meters - this is a big triangle!

In [ ]:
verts = [[[0.57735027, -5.77350269e-01, -0.57735027],
         [0.57735027, 5.77350269e-01, -0.57735027],
         [-0.57735027, 5.77350269e-01, -0.57735027]]]
# Note: initialization using array of vertices

# load our vertices into a UXarray Grid object
vgrid = ux.Grid(verts, vertices=True,
                        islatlon=False,
                        concave=False)

# Note the incorrect units for the x and y coordinates (expand the cell Mesh2_node_x etc.
# See units are set as degree_east and degree_north)
vgrid.ds

In [ ]:
vgrid.calculate_total_face_area()

The above result (0.0002) is WRONG as the units are assumed to be spherical, when the coordinates are in cartesian system

In [ ]:
# Set correct units for the x and y coordinates
vgrid.Mesh2_node_x.attrs["units"] = "m"
vgrid.Mesh2_node_y.attrs["units"] = "m"
vgrid.Mesh2_node_z.attrs["units"] = "m"

# It is important to set the units for the x and y coordinates
# otherwise the area will be incorrect, as the units are assumed to be spherical by default
# Without the above setting the coordinates will be assumed to be in spherical coordinates
# and the area will be calculated incorrectly

# Calculate the area of the triangle
area_gaussian = vgrid.calculate_total_face_area(
            quadrature_rule="gaussian", order=5)
area_gaussian

## 5. Calculate area of multiple vertices in spherical coordinates

In [ ]:
faces_verts_ndarray = np.array([
    np.array([[150, 10, 0], [160, 20, 0], [150, 30, 0], [135, 30, 0], [125, 20, 0],
              [135, 10, 0]]),
    np.array([[125, 20, 0], [135, 30, 0], [125, 60, 0], [110, 60, 0], [100, 30, 0],
              [105, 20, 0]]),
    np.array([[95, 10, 0], [105, 20, 0], [100, 30, 0], [85, 30, 0], [75, 20, 0],
              [85, 10, 0]]),
])

In [ ]:
# load our vertices into a UXarray Grid object
verts_grid = ux.Grid(faces_verts_ndarray, vertices=True,
                        islatlon=True,
                        concave=False)
verts_grid.ds

In [ ]:
verts_grid.face_areas

# 6. Calculate area with points, without loading a UXarray grid

create 3 python array variables cart_x, cart_y and cart_z forming a quad face


In [ ]:
cart_x = [
   0.577340924821405, 0.577340924821405, 0.577340924821405,
   0.577340924821405
]
cart_y = [
   0.577343045516932, 0.577343045516932, -0.577343045516932,
   -0.577343045516932
]
cart_z = [
   0.577366836872017, -0.577366836872017, 0.577366836872017,
   -0.577366836872017
]


Call `helpers.calculate_face_area` fuction and set the coordinates type as "cartesian", default is set to spherical.

Note: This function only takes in numpy array as input. UXarray Grid object is not involved in the calculation below:

In [ ]:
ux.helpers.calculate_face_area(np.array(cart_x), np.array(cart_y), np.array(cart_z), coords_type="cartesian")